In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 

In [3]:
base_path = '../input/'

In [2]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('../input/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [5]:
ssubm

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75.0,75.0
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,0,75.0,75.0


In [8]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/'+building+'/*')))
    print(building)
    
    wifi = list()
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        
        for file in files:
            with open(file) as f:
                txt = f.readlines()
                
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 1000].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    # del df
    # del wifi
    gc.collect()
    
    break
    
# TYPE_WIFI	ssid	bssid	RSSI	frequency	last seen timestamp	

5a0546857ecc773753327266
941


In [19]:
for gid, g in df.groupby(0):
    g = g.drop_duplicates(subset=3)
    break

In [24]:
g

,0,1,2,3,4,5,6
0,1569655291181,TYPE_WIFI,7182afc4e5c212133d5d7d76eb3df6c24618302b,0e5a262402b8c9d98a3b5a424863f6a6c655ab98,-53,5825,1569655290412
1,1569655291181,TYPE_WIFI,b7e6027447eb1f81327d66cfd3adbe557aabf26c,8b3c0b4adecdc3281c15a824690f5bdf8563ad82,-53,5825,1569655290411
2,1569655291181,TYPE_WIFI,78f32ce289cdaccfc83a01ef3cbf4af52942c063,14bc29d59e0046cc7d5ce53054f702da18f37c53,-53,5825,1569655290413
3,1569655291181,TYPE_WIFI,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,688a2ecf1f543651ec563d2c397b137baf8d39ea,-54,5825,1569655290410
4,1569655291181,TYPE_WIFI,5ec848d2a1cf52ae0eb73b148e7582d539efbe7f,12f40bbe5cffaf37e3225bdeb8a8526b76e7d59f,-54,2437,1569655289383
...,...,...,...,...,...,...,...
381,1569655291181,TYPE_WIFI,da39a3ee5e6b4b0d3255bfef95601890afd80709,d5982e6e5110e61911f44063ef6befbe84da7a9d,-90,5745,1569655270745
382,1569655291181,TYPE_WIFI,78f32ce289cdaccfc83a01ef3cbf4af52942c063,b769f60d8499d23acab612b72eae2b7b984b09cf,-90,5805,1569655288056
383,1569655291181,TYPE_WIFI,b7e6027447eb1f81327d66cfd3adbe557aabf26c,524adb9add1079ddd0428c596143e33977e4e36d,-90,5785,1569655277328
384,1569655291181,TYPE_WIFI,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,fc230ca97bc6bd2969b2194cc896000e972d5542,-90,5745,1569655287889


In [5]:
with open("bssid_1000.json", "w") as f:
    json.dump(bssid, f)

with open("bssid_1000.json") as f:
    bssid = json.load(f)

In [17]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        print(floor)
        for file in files:
            wifi = list()
            waypoint = list()
            
            with open(file) as f:
                txt = f.readlines()
            
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)

            df = pd.DataFrame(np.array(wifi))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).replace(np.nan, -999).T
                
                # 对于一条记录，xyf，最近时刻商场内热门wifi对应的强度
                feat["x"] = float(waypoint[nearest_wp_index][2])
                feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_1000_train.csv")
    
    break

5a0546857ecc773753327266
-1
0
1
2
3


In [18]:
building_df
# 训练集：对于每个商场，941个wifi的信号来作为特征， x/y/f

3,000840e5c600de293cea57f13326f273c86c3988,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00f0904087c01d922d6ebf3005607dfdeaf6687b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01e25e4a25acd32baf5137b3031151f751fadbb4,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,...,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe3211f90e4ab1f500e10fe175ae6142f4b13130,ffa41c79865d7fb336f586e0dec8b080db1027fb,x,y,f,path
4,-999,-999,-84,-999,-77,-999,-999,-999,-76,-999,...,-999,-999,-999,-999,-999,-999,162.331820,110.822685,-1,5e158f081506f2000638fd29
4,-999,-999,-84,-999,-81,-999,-999,-999,-76,-999,...,-999,-999,-999,-999,-999,-999,163.386870,115.330850,-1,5e158f081506f2000638fd29
4,-999,-999,-84,-999,-81,-999,-999,-999,-81,-999,...,-999,-999,-999,-999,-999,-999,163.386870,115.330850,-1,5e158f081506f2000638fd29
4,-999,-999,-84,-999,-81,-999,-999,-999,-81,-999,...,-999,-999,-999,-999,-999,-999,164.344310,119.490100,-1,5e158f081506f2000638fd29
4,-999,-999,-84,-999,-72,-999,-999,-999,-81,-999,...,-999,-999,-999,-999,-999,-999,164.344310,119.490100,-1,5e158f081506f2000638fd29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,-999,-68,-999,-999,-999,-999,-999,-88,-999,-78,...,-73,-78,-89,-999,-999,-999,116.952774,110.410630,3,5d79bdfe43d77a0006b63a5d
4,-999,-68,-999,-999,-999,-999,-999,-88,-999,-78,...,-73,-86,-89,-999,-999,-999,116.952774,110.410630,3,5d79bdfe43d77a0006b63a5d
4,-999,-68,-999,-999,-999,-999,-999,-88,-999,-78,...,-73,-80,-89,-999,-999,-999,116.952774,110.410630,3,5d79bdfe43d77a0006b63a5d
4,-72,-999,-999,-999,-999,-999,-999,-79,-999,-999,...,-71,-86,-88,-999,-999,-999,184.550810,100.736580,3,5d8f0a52d5bae80006eb8de1


In [7]:
# Generate the features for the test set

ssubm_building_g = ssubm_df.groupby(0)
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    break
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
    print(gid0)
    for gid,g in g0.groupby(1):

        # get all wifi time locations, 
        with open(os.path.join(base_path, 'test/' + g.iloc[0,1] + '.txt')) as f:
            txt = f.readlines()

        wifi = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)

        wifi_df = pd.DataFrame(wifi)
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist())
        
        for timepoint in g.iloc[:,2].tolist():

            deltas = (wifi_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_1000_test.csv")
    feature_dict[gid0] = feature_df